In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torchvision
import cv2

from torch.utils.data import DataLoader
from dataset.carvana_dataset import *
from dataset import carvana_dataset
from utils.scripts import *
from model.link_net import LinkNet18
from model.loss import BCEDiceLoss

torch.manual_seed(42)

C:\Users\vanou\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
DATA_DIR = Path('input')
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR = DATA_DIR / 'test'
TRAIN_MASKS = DATA_DIR / 'train_masks'

In [3]:
print(f'Train images = {len(os.listdir(TRAIN_DIR))}')
print(f'Test images = {len(os.listdir(TEST_DIR))}')
print(f'Train masks = {len(os.listdir(TRAIN_MASKS))}')

Train images = 5088
Test images = 100064
Train masks = 5088


In [4]:
train_df, val_df = get_train_val(DATA_DIR/'train_masks.csv')

In [5]:
train_df

,img,rle_mask
0,28d7fb5ba432_08.jpg,814382 7 816298 9 818214 12 820131 13 822047 1...
1,1e6f48393e17_11.jpg,829441 89 831332 137 833232 174 835136 202 837...
2,9cc257b449d0_15.jpg,643334 1 645252 2 647170 2 649088 2 651006 3 6...
3,b24fd9084449_01.jpg,543758 1 545675 3 547593 3 549511 3 551429 3 5...
4,7ac210ba75a1_03.jpg,630242 10 631941 68 632157 16 633836 119 63407...
...,...,...
4066,4e308ad8a254_08.jpg,534206 3 536124 3 538041 4 539959 4 541877 3 5...
4067,c87688f6960e_05.jpg,515317 3 517234 4 519152 5 521070 4 522987 5 5...
4068,e597d76a0c33_03.jpg,760518 97 762411 145 764310 181 766214 211 768...
4069,dd47eb7ac4ee_09.jpg,898742 1 900465 5 900659 5 902352 90 902576 8 ...


In [6]:
val_df

,img,rle_mask
0,28d7fb5ba432_08.jpg,814382 7 816298 9 818214 12 820131 13 822047 1...
1,1e6f48393e17_11.jpg,829441 89 831332 137 833232 174 835136 202 837...
2,9cc257b449d0_15.jpg,643334 1 645252 2 647170 2 649088 2 651006 3 6...
3,b24fd9084449_01.jpg,543758 1 545675 3 547593 3 549511 3 551429 3 5...
4,7ac210ba75a1_03.jpg,630242 10 631941 68 632157 16 633836 119 63407...
...,...,...
1012,6e016b8b3617_06.jpg,789606 11 791518 18 793091 119 793432 22 79498...
1013,6cc98271f4dd_07.jpg,551430 67 551645 8 553322 143 553560 12 555222...
1014,dd47eb7ac4ee_08.jpg,898769 19 900672 56 902415 27 902584 75 904308...
1015,bb7625a3f1d4_06.jpg,641913 1 643830 2 645747 3 647664 3 649581 3 6...


In [7]:
transforms = get_transform(True)
train_dataset = CarvanaDataset(train_df, TRAIN_DIR,
                              transform=transforms)
val_dataset = CarvanaDataset(val_df, TRAIN_DIR,
                              transform=transforms)

train_loader = DataLoader(train_dataset, batch_size=16,
                           shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16,
                           shuffle=False, num_workers=2)

In [8]:
model = LinkNet18(num_classes=1)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

LinkNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (encoder1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tr

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.09)
criterion = BCEDiceLoss()

In [ ]:
epochs = 50
best_loss = np.inf
for epoch in range(epochs):
    print(f"Epoch [{epoch+1}/{epochs}]")
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        image, mask = batch['image'].to(device), batch['mask'].to(device)
        logits = model(image)
        loss = criterion(logits, mask)
        
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.)
        optimizer.step()
        
        running_loss += loss.item()
    
    mean_loss = round(running_loss / len(train_loader), 3)     
    print(f"Total loss = {mean_loss}")
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            image, mask = batch['image'].to(device), batch['mask'].to(device)
            logits = model(image)
            loss = criterion(logits, mask)

            val_loss += loss.item()
        
    mean_val_loss = round(val_loss / len(val_loader), 3)
    print(f"Total val_loss = {mean_val_loss}")
    
    scheduler.step(mean_val_loss)
    
    if best_loss > mean_val_loss:
        best_loss = mean_val_loss
        PATH = f'model/linknet18_epoch{epoch}_val_loss{best_loss}.pth'
        torch.save(model.state_dict(), PATH)
        

Epoch [1/50]


C:\Users\vanou\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Total loss = 1.05
Total val_loss = 0.999
Epoch [2/50]


C:\Users\vanou\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Total loss = 0.987
Total val_loss = 0.984
Epoch [3/50]
Total loss = 0.984
Total val_loss = 0.984
Epoch [4/50]
Total loss = 0.984
Total val_loss = 0.984
Epoch [5/50]
Total loss = 0.984
Total val_loss = 0.984
Epoch [6/50]
Total loss = 0.984
Total val_loss = 0.983
Epoch [7/50]
Total loss = 0.984
Total val_loss = 0.984
Epoch [8/50]
